In [4]:
%matplotlib inline
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.optim as optim
from torchvision import transforms,models
import time

In [5]:
vggList = [models.vgg11(pretrained=True).features,models.vgg13(pretrained=True).features,models.vgg16(pretrained=True).features,models.vgg19(pretrained=True).features]
devices=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(devices)

for vgg in vggList:
    
    for param in vgg.parameters():
        param.requires_grad_(False)
        
    vgg.to(devices)


cpu


In [10]:
def load_image(img_path,max_size=400,shape=None):
    image=Image.open(img_path).convert('RGB')
    if max(image.size) > max_size:
        size=max_size
    else:
        size=max(image.size)
    if shape is not None:
        size=shape
    
    in_transform=transforms.Compose([transforms.Resize(size),transforms.ToTensor(),transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))])
    
    image = in_transform(image)[:3,:,:].unsqueeze(0)
    return image

path = "images/"
contentListName = ['space_needle.jpg','space_needle.jpg','space_needle.jpg']
contentList = list()
styleListName = ['space_needle.jpg','space_needle.jpg','space_needle.jpg']
styleList = list()

for name in contentListName:
    
    contentList.append(load_image(path + name).to(devices))
    
for name in styleListName:
    
    styleList.append(load_image(path + name).to(devices))
    #styleList.append(load_image(path + name,shape=content.shape[-2:]).to(devices))


In [7]:
def style_loss(Y_hat, gram_Y):
    return np.square(gram(Y_hat) - gram_Y).mean()

def content_loss(Y_hat, Y):
    return np.square(Y_hat - Y).mean()

def gram(X):
    num_channels, n = X.shape[1], X.size // X.shape[1]
    X = X.reshape(num_channels, n)
    return np.dot(X, X.T) / (num_channels * n)

def compute_loss(X, contents_Y_hat, styles_Y_hat, contents_Y, styles_Y_gram):
    # Calculate the content, style, and total variance losses respectively
    contents_l = [content_loss(Y_hat, Y) * content_weight for Y_hat, Y in zip(
        contents_Y_hat, contents_Y)]
    styles_l = [style_loss(Y_hat, Y) * style_weight for Y_hat, Y in zip(
        styles_Y_hat, styles_Y_gram)]
    tv_l = tv_loss(X) * tv_weight
    # Add up all the losses
    l = sum(styles_l + contents_l + [tv_l])
    return contents_l, styles_l, tv_l, l

In [ ]:
def create(vgg,target,convLayerKey,styleWeights,contentFeatures,steps=2000,showEvery=400,showTime=true,showLoss=true):
    
    if(show_time):
        start = time.time()
    
    optimizer=optim.Adam([target],lr=0.003)
    
    for ii in range(1,steps+1):
        targetFeatures=get_features(target,vgg)
        content_loss=torch.mean((target_features[convLayerKey] - contentFeatures[convLayerKey])**2)

        style_loss=0
        for layer in styleWeights:
            target_feature=targetFeatures[layer]
            target_gram=gram_matrix(target_feature)
            _,d,h,w=target_feature.shape
            style_gram=style_grams[layer]
            layerStyleLoss=styleWeights[layer]*torch.mean((target_gram-style_gram)**2)
            style_loss+=layerStyleLoss/(d*h*w)

        total_loss=content_weight*content_loss+style_weight*style_loss

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        if ii% show_every==0:
            if ();
                print('Style loss:',style_loss)
                print('Total loss:',total_loss.item())
            plt.imshow(im_convert(target))
            plt.show()
            if(show_time):
                print('Time:',time.time() - start)
                
    if(show_time):            
        print(time.time() - start)
    return 

# vgg11

# vgg13

# vgg16

# vgg19